In [ ]:
# MelNet  -> Implemented using tensorflow : https://arxiv.org/pdf/1906.01083.pdf
# Strongly advice to read paper to understand code.
# Only does Unconditional generation at the moment
# Further updates for conditional generation later
#%tensorflow 1
from tensorflow.keras.layers import LSTM,GRU, Dense, Input, Lambda, BatchNormalization, Input, Concatenate
#from keras.layers import Conv2D+
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import tensorflow_probability as tfp
import gc
import math
from tqdm import tqdm
import numpy as np

In [ ]:
import math
import json
import time
import gc

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 10.6 GB  | Proc size: 3.3 GB
GPU RAM Free: 502MB | Used: 14577MB | Util  97% | Total 15079MB


In [ ]:
  tfd =  tfp.distributions #Access to different statistical distributions. We use the tf.MixtureSameFamily

Main MelNet Function

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

#Create FrequencyDelayedStack and TimeDelayedStack functions

In [ ]:
DESTINATION_PATH= '/content/drive/MyDrive/Deep Learning/MelNet Implementation/Training data'
DATA_PATH = '/content/drive/MyDrive/Deep Learning/MelNet Implementation/output'

In [ ]:
'''
  MelNet is an autoregressive (audio generative) model that takes mel spectogram as input and outputs the distribution parameters (mean,std) responsible for 
  the input. It models each input using a gaussian mixture model. It maximizes the marginal likelihood of the input samples.
'''

def gaussian_mixture_loss(label_spect, mu , std , alpha):
  '''Calculates the loss by sampling from the distribution parameterized by the model output. It takes the model input and output(mu,std,alpha)
  and calculates the loss.'''
  # Dimensions: (batch_size, dim_f1, dim_t1, num of mixture)

  # mu = tensor_out[:, 1, :]
  # sigma = tensor_out[:, 2, :]
  # alpha = tensor_out[:, 0, :]

  gm = tfd.MixtureSameFamily(
      mixture_distribution = tfd.Categorical(
          probs=alpha),
          components_distribution = tfd.Normal(loc = mu, scale = std))
  
  #log_loss = -tf.reduce_sum(gm.log_prob(tf.squeeze(label_spect)))
  log_loss = -tf.math.reduce_mean(gm.log_prob(label_spect))

  return log_loss


def sampling_gauss(mu, std , alpha):
  # Function to sample from distribution
  # Dimensions: (batch_size, dim_f1, dim_t1, num of mixture)
  gm = tfd.MixtureSameFamily(
      mixture_distribution = tfd.Categorical(
          probs=alpha), components_dsitribution=tfd.Normal(loc = mu, scale = std))
  
  return gm


In [ ]:
def sample_from_disribution(mu , std, alpha ,batch_size=1):
    # Sampling from a gaussian mixture model:
    
    dim_f = np.size(miut1, 1)
    dim_t = np.size(miut1, 2)
    #batch_size = np.size(miut1, 0)
    num_mixture = int(np.size(miut1, 3))
    # data dimensions : ( Batch_size , dim_f , dim_t,3k )
    # Apply transformation as in (3) (4) (5) to gaussian model:
    #output_tot = np.zeros((dim_f, dim_t))
    output_tot_batch_format = np.zeros((batch_size, dim_f, dim_t))

    for batch_num in range(batch_size):
        sum_mixture = 0
        alpha_batch = alphat1[batch_num]
        miu_batch = miut1[batch_num]
        sigma_batch = sigmat1[batch_num]
        sum_alpha_batch = np.sum(alpha_batch, axis=2)
        
        out = np.zeros((dim_f, dim_t))
        if np.sum(1) > 0:
            for i in range(dim_f):
                for j in range(dim_t):
                    ind = np.random.choice(np.arange(0, num_mixture), p=np.ravel([alpha_batch[i, j, :]]))
                    out[i, j] = np.random.normal(miu_batch[i, j, ind], sigma_batch[i, j, ind])

        #output_tot = np.append(output_tot, out, axis=1)
        output_tot_batch_format = np.append(output_tot_batch_format, np.expand_dims(out, axis=0), axis=0)
    return output_tot_batch_format



 10s, 22050hz   = 861 samples   431 samples

 15s, 22050hz   = 1291 samples  646 samples

 20s,22050hz   = 1724 samples   862 samples

 25s, 22050hz  = 2154 samples   1077 samples

 30s,18000hz  =  2110 samples   1055 samples

 30s, 22050hz  =  2584 samples  1292 samples
 35s, 22050hz =  3015 samples   1507 samples
 

 45s, 18000hz  =  3184 samples  1582 samples

45s, 22050hz  = 3876 samples,  1938 samples

 1 min,18000hz  = 4220 samples  2109 samples
 
 1 min, 22050hz  = 5168 samples 2583 samples


  # when we perform stft, the number of time frames we get is:
        # self.T = int(hp.audio.sr * hp.audio.duration) // hp.audio.hop_length + 1
        # 10*22050 // 256 + 1 = 862 (blizzard)
        # 6*22050 // 256 + 1 = 517 (maestro)
        # 6*16000 // 180 + 1 = 534 (voxceleb2)
        # 10*16000 // 180 + 1 = 889 (tedlium3)

In [ ]:
class DelayedLSTMAdd(Model):
  def __init__(self,units,dim_f,dim_t):
    super(DelayedLSTMAdd,self).__init__()
    self.w_t = Dense(units)
    self.w_f = Dense(units)

    self.t_forward = LSTM(units, input_shape=(None, dim_t, units), return_state=True, 
                          return_sequences=True,)
    
    self.f_up = LSTM(units, input_shape=(None, dim_f, units), return_state=True,
                      return_sequences=True)
    
    self.f_down = LSTM(units, input_shape=(None, dim_f, units), return_state=True, 
                        return_sequences=True)
    
    self.freqstack = LSTM(units, input_shape=(None, dim_f, units), return_state=True, 
                           return_sequences=True)
    

  def call(self,input_t, input_f):
    # Extract dimension details.
    B, F, T, H = input_t.shape

    # reshape time input for parallelization and reshape to initial shape
    output_t = tf.reshape(input_t,(-1, T, H))
    output_t = self.t_forward(output_t)[0]
    output_t = tf.reshape(output_t,(B,F,T,H))

    # transpose input to freq axis, reverse for the reverse lstm
    # To save memory, i reuse variables to store operation results.
    f_axis = tf.transpose(input_t,[0,2,1,3])
    f_axis = tf.reshape(f_axis,(-1,F,H))
    f_axis = self.f_up(f_axis)[0]
    f_axis = tf.transpose(tf.reshape(f_axis, (B,T,F,H)), [0,2,1,3])
    output_t = tf.add(output_t,f_axis)

    #Transpose and reverse frequency axis before running through lstm layer.
    f_axis = tf.reverse(tf.transpose(input_t, [0,2,1,3]), [2])    
    f_axis = tf.reshape(f_axis, (-1,F,H))
    f_axis = self.f_down(f_axis)[0]
    f_axis = tf.transpose(tf.reshape(f_axis, (B,T,F,H)),[0,2,1,3])

    output_t = tf.add(output_t, f_axis)
    output_t  = input_t + self.w_t(output_t)

    # Add time lstm output to freq input
    f = input_f + output_t

    #reshape frequency input.
    f = tf.transpose(f, [0,2,1,3])
    f = tf.reshape(f, [-1,F,H])
    f = self.freqstack(f)[0]
    f = tf.transpose(tf.reshape(f, [B,T,F,H]), [0,2,1,3])
    f = input_f  + self.w_f(f)

    return output_t , f




class FeatureExtractorAdd(Model):
  def __init__(self,units,dim_f,dim_t):
    super(FeatureExtractorAdd,self).__init__()
    self.w = Dense(units)
    self.t_forward = LSTM(units, input_shape=(None, dim_t, units), return_state=True, 
                           return_sequences=True,)
    
    self.f_up = LSTM(units, input_shape=(None, dim_f, units), return_state=True,
                     return_sequences=True)
    
    self.f_down = LSTM(units, input_shape=(None, dim_f, units), return_state=True, 
                       return_sequences=True)
    
    self.t_backward = LSTM(units, input_shape=(None, dim_t, units), return_state=True, 
                          return_sequences=True)
    

  def call(self,input_t):
    # Returns hidden_state of condition.
    # Extract dimension details.
    B, F, T, H = input_t.shape
    #print('input', input_t.shape)

    # Foward Time axis
    # reshape time input for parallelization and reshape to initial shape
    hidden = tf.reshape(input_t,(-1, T, H))
    hidden = self.t_forward(hidden)[0]
    #print('After', hidden.shape)
    hidden = tf.reshape(hidden,(B,F,T,H))
    
    # Reverse Time axis
    # Reuse the features variable
    features = tf.reverse(input_t, [2])
    features = tf.reshape(features, (-1, T, H))
    features = self.t_backward(features)[0]
    features = tf.reshape(features ,(B,F,T,H))
    hidden = tf.add(hidden , features)

    # transpose input to freq axis, reverse for the reverse lstm
    # Reuse features variable
    features = tf.transpose(input_t,[0,2,1,3])
    features = tf.reshape(features,(-1,F,H))
    features = self.f_up(features)[0]
    features = tf.transpose(tf.reshape(features, (B,T,F,H)), [0,2,1,3])
    hidden = tf.add(hidden,features)


    features = tf.reverse(tf.transpose(input_t, [0,2,1,3]), [2])
    features = tf.reshape(features, (-1,F,H))
    features = self.f_down(features)[0]
    features = tf.transpose(tf.reshape(features, (B,T,F,H)),[0,2,1,3])
    hidden = tf.add(hidden,features)

    # Add residual connection
    hidden  = input_t + self.w(hidden)

    return hidden


In [ ]:
class DelayedLSTM(Model):
  def __init__(self,units,dim_f,dim_t):
    super(DelayedLSTM,self).__init__()
    self.w_t = Dense(units)
    self.w_f = Dense(units)

    self.t_forward = LSTM(units, input_shape=(None, dim_t, units), return_state=True, 
                           return_sequences=True,)
    
    self.f_up = LSTM(units, input_shape=(None, dim_f, units), return_state=True,
                      return_sequences=True)
    
    self.f_down = LSTM(units, input_shape=(None, dim_f, units), return_state=True, 
                        return_sequences=True)
    
    self.freqstack = LSTM(units, input_shape=(None, dim_f, units), return_state=True, 
                          return_sequences=True)
    

  def call(self,input_t, input_f):
    # Extract dimension details.
    B, F, T, H = input_t.shape

    # reshape time input for parallelization and reshape to initial shape
    output_t = tf.reshape(input_t,(-1, T, H))
    output_t = self.t_forward(output_t)[0]
    output_t = tf.reshape(output_t,(B,F,T,H))

    # transpose input to freq axis, reverse for the reverse lstm
    # To save memory, i reuse variables to store operation results.
    f_axis = tf.transpose(input_t,[0,2,1,3])
    f_axis = tf.reshape(f_axis,(-1,F,H))
    f_axis = self.f_up(f_axis)[0]
    f_axis = tf.transpose(tf.reshape(f_axis, (B,T,F,H)), [0,2,1,3])
    output_t = tf.concat([output_t,f_axis], axis=-1)

    #Transpose and reverse frequency axis before running through lstm layer.
    f_axis = tf.reverse(tf.transpose(input_t, [0,2,1,3]), [2])    
    f_axis = tf.reshape(f_axis, (-1,F,H))
    f_axis = self.f_down(f_axis)[0]
    f_axis = tf.transpose(tf.reshape(f_axis, (B,T,F,H)),[0,2,1,3])

    output_t = tf.concat([output_t, f_axis], axis=-1)
    output_t  = input_t + self.w_t(output_t)

    # Add time lstm output to freq input
    f = input_f + output_t

    #reshape frequency input.
    f = tf.transpose(f, [0,2,1,3])
    f = tf.reshape(f, [-1,F,H])
    f = self.freqstack(f)[0]
    f = tf.transpose(tf.reshape(f, [B,T,F,H]), [0,2,1,3])
    f = input_f  + self.w_f(f)

    return output_t , f


class FeatureExtractor(Model):
  def __init__(self,units,dim_f,dim_t):
    super(FeatureExtractor,self).__init__()
    self.w = Dense(units)
    self.t_forward = LSTM(units, input_shape=(None, dim_t, units), return_state=True, 
                           return_sequences=True,)
    
    self.f_up = LSTM(units, input_shape=(None, dim_f, units), return_state=True,
                    return_sequences=True)
    
    self.f_down = LSTM(units, input_shape=(None, dim_f, units), return_state=True, 
                       return_sequences=True)
    
    self.t_backward = LSTM(units, input_shape=(None, dim_t, units), return_state=True, 
                           return_sequences=True)
    

  def call(self,input_t):
    # Returns hidden_state of condition.
    # Extract dimension details.
    B, F, T, H = input_t.shape
  

    # Foward Time axis
    # reshape time input for parallelization and reshape to initial shape
    hidden = tf.reshape(input_t,(-1, T, H))
    hidden = self.t_forward(hidden)[0]
    #print('After', hidden.shape)
    hidden = tf.reshape(hidden,(B,F,T,H))
    
    # Reverse Time axis
    # Reuse the features variable
    features = tf.reverse(input_t, [2])
    features = tf.reshape(features, (-1, T, H))
    features = self.t_backward(features)[0]
    features = tf.reshape(features ,(B,F,T,H))
    hidden = tf.concat([hidden , features], axis=-1)

    # transpose input to freq axis, reverse for the reverse lstm
    # Reuse features variable
    features = tf.transpose(input_t,[0,2,1,3])
    features = tf.reshape(features,(-1,F,H))
    features = self.f_up(features)[0]
    features = tf.transpose(tf.reshape(features, (B,T,F,H)), [0,2,1,3])
    #print(features)
    hidden = tf.concat([hidden,features], axis=-1)


    features = tf.reverse(tf.transpose(input_t, [0,2,1,3]), [2])
    features = tf.reshape(features, (-1,F,H))
    features = self.f_down(features)[0]
    features = tf.transpose(tf.reshape(features, (B,T,F,H)),[0,2,1,3])
    #print(features)
    hidden = tf.concat([hidden,features], axis=-1)

    # Add residual connection
    hidden  = input_t + self.w(hidden)
    gc.collect()

    return hidden

In [ ]:
print(gc.get_threshold())
print(gc.get_count())

(700, 10, 10)
(44, 5, 8)


In [ ]:
class TierAdd(Model):
  def __init__(self,units,dim_f,dim_t,base_tier=True,layers=6,gmm=10):
    super(TierAdd,self).__init__()
    self.dims = dim_f, dim_t
    if base_tier:
       self.W_t_0 = Dense(units)
       self.W_f_0 = Dense(units)
       #self.W_c_0 = nn.Linear(freq, num_hidden)
       self.tier_layers = [
          DelayedLSTMAdd(units, dim_f, dim_t) for _ in range(layers)
      ]
    else:
      self.feature_layers = [FeatureExtractorAdd(units,dim_f,dim_t) for _ in range(layers) ]
      self.tier_layers = [ DelayedLSTMAdd(units,dim_f,dim_t) for _ in range(layers)]

      self.W_t_0 = Dense(units)
      self.W_f_0 = Dense(units)
      self.W_feature = Dense(units)
      self.W_t_ft = Dense(units)
      self.W_f_ft = Dense(units)

    self.K = gmm
    # Linear map freqstack_output to produce unconstrained Gaussian Mixture Model parameters 
    self.Wt_theta = Dense(3*self.K)
    

  def call(self, tier_input,hidden=None):
    if type(hidden) != None:
      hidden = self.W_feature(tf.expand_dims(hidden, axis=-1 ))
      for layer in self.feature_layers:
        hidden = layer(hidden)

      # Define timestack input:  pad input
      # Add weighted conditions to it.
      output_time = self.W_t_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=2),axis=-1)) + self.W_t_ft(hidden)
      # Define freqstack input: shift input
      output_freq = self.W_f_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=1),axis=-1)) + self.W_f_ft(hidden)
      # output_time = output_time + self.W_t_ft(hidden)
      # output_freq = output_freq + self.W_f_ft(hidden)
      del hidden   # To save memory, reassign hidden

    else:
      # Define timestack input:  pad input
      output_time = self.W_t_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=2),axis=-1))
      # Define freqstack input: shift input
      output_freq = self.W_f_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=1),axis=-1))


    count = 0
    for layer in self.tier_layers:
      output_time, output_freq = layer(output_time, output_freq) 
      count +=1

    theta_hat = self.Wt_theta(output_freq)

    # Calculating mixture gaussian parameters (unconstrained) mu (mean), sigma(std), alpha (probability density fraction):

    mu = theta_hat[..., :self.K] 
    std = theta_hat[..., self.K:2*self.K]
    alpha = theta_hat[..., 2*self.K:]

    # Convert unconstrained to constrained parameters
    std = K.exp(std)
    alpha = K.exp(alpha)
    alpha = alpha[...,:] / tf.expand_dims(K.sum(alpha, axis=-1),axis=-1)
    # mixture sum to one: Alpha coefficients for all gaussian distributions must sum to one 
          
    return mu, std, alpha


In [ ]:
class Tier(Model):
  def __init__(self,units,dim_f,dim_t,base_tier=True,layers=6,gmm=10):
    super(Tier,self).__init__()
    self.dims = dim_f, dim_t
    if base_tier:
       self.W_t_0 = Dense(units)
       self.W_f_0 = Dense(units)
       #self.W_c_0 = nn.Linear(freq, num_hidden)
       self.tier_layers = [
          DelayedLSTM(units, dim_f, dim_t) for _ in range(layers)
      ]
    else:
      self.feature_layers = [FeatureExtractor(units,dim_f,dim_t) for _ in range(layers) ]
      self.tier_layers = [ DelayedLSTM(units,dim_f,dim_t) for _ in range(layers)]

      self.W_t_0 = Dense(units)
      self.W_f_0 = Dense(units)
      self.W_feature = Dense(units)
      self.W_t_ft = Dense(units)
      self.W_f_ft = Dense(units)

    self.K = gmm
    # Linear map freqstack_output to produce unconstrained Gaussian Mixture Model parameters 
    self.Wt_theta = Dense(3*self.K)
    

  def call(self, tier_input,hidden=None):
    if type(hidden) != None:
      hidden = self.W_feature(tf.expand_dims(hidden, axis=-1 ))
      for layer in self.feature_layers:
        hidden = layer(hidden)

      # Define timestack input:  pad input
      # Add weighted conditions to it.
      output_time = self.W_t_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=2),axis=-1)) + self.W_t_ft(hidden)
      # Define freqstack input: shift input
      output_freq = self.W_f_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=1),axis=-1)) + self.W_f_ft(hidden)
      # output_time = output_time + self.W_t_ft(hidden)
      # output_freq = output_freq + self.W_f_ft(hidden)
      del hidden   # To save memory, reassign hidden

    else:
      # Define timestack input:  pad input
      output_time = self.W_t_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=2),axis=-1))
      # Define freqstack input: shift input
      output_freq = self.W_f_0(tf.expand_dims(tf.roll(tier_input,shift=1, axis=1),axis=-1))


    count = 0
    for layer in self.tier_layers:
      output_time, output_freq = layer(output_time, output_freq) 
      count +=1

    theta_hat = self.Wt_theta(output_freq)

    # Calculating mixture gaussian parameters (unconstrained) mu (mean), sigma(std), alpha (probability density fraction):

    mu = theta_hat[..., :self.K] 
    std = theta_hat[..., self.K:2*self.K]
    alpha = theta_hat[..., 2*self.K:]

    # Convert unconstrained to constrained parameters
    std = K.exp(std)
    alpha = K.exp(alpha)
    alpha = alpha[...,:] / tf.expand_dims(K.sum(alpha, axis=-1),axis=-1)
    # mixture sum to one: Alpha coefficients for all gaussian distributions must sum to one 

    gc.collect()
          
    return mu, std, alpha



In [ ]:
import pickle

def recursive_split_save(data, dest_path, axes, filenames, n_tiers):
  axis = axes.pop()
  even , odd  = split(data, axis= axis)
  # save even and recurse on odd
  if len(axes) == 0:
    save(even,  odd, dest_path, filenames, n_tiers)
    return
  else:
    return recursive_split_save(odd, dest_path, axes, filenames, (n_tiers - 1))

def save(even, odd, dest_path, filenames , n_tiers):
  for i in range(odd.shape[0]):
    file_sample_path = f'Tier {n_tiers}/samples/{filenames[i]}'
    file_feature_path = f'Tier {n_tiers}/features/{filenames[i]}'

    with open(os.path.join(dest_path, file_sample_path), 'wb') as f:
      pickle.dump(even[i], f)

    with open(os.path.join(dest_path,file_feature_path), 'wb') as f:
      pickle.dump(odd[i],f)

def get_splits(mel, time):
  splits = []
  for i in range(len(time)):
    if i >= len(mel):
      splits.append(time[i])
    else:
      splits.append(mel[i])
      splits.append(time[i])
  return splits


class MelNet(Model):
  def __init__(self, units=512 , gmm=10 , mel =256, audio_length=30, sr=22050, window=256, base_tier_dim=(32,50), conditional=False):
    super(MelNet,self).__init__()
    self.mel_dim = mel
    self.base_tier_dim = base_tier_dim
    self.conditional = conditional
    self.load_tiers(units, gmm, audio_length, sr, window)

  def cal_each_tier_dim(self,audio_length, sr=22050, window=256):
    # Calculates the dimensions of each tier that makes up the overall melnet model.
    self.x_samples = round(audio_length * sr / window ) #+ 1
    x_samples = self.x_samples
    mel_samples = self.mel_dim
    self.mel_splits = []
    self.tiers_t_dim=[]
    self.time_splits = []
    self.tiers_f_dim = []

    while (x_samples/2) >= self.base_tier_dim[1]: # while not base tier dim, append 1 or 0 (depending on axis) to list and divide n_of_samples by 2 for each axis
      self.time_splits.append(1)
      x_samples = x_samples / 2
      self.tiers_t_dim.append(x_samples)

    while (mel_samples /2) >= self.base_tier_dim[0]:
      self.mel_splits.append(0)
      mel_samples = mel_samples/2
      self.tiers_f_dim.append(mel_samples)

    self.tiers_f_dim.reverse()
    self.tiers_t_dim.reverse()
    #print(self.tiers_f_dim, self.tiers_t_dim)


  def call(self, tier_num, x, features=None, batch_size = 16,path='./checkpoints/train'):
    # Define optimizer
    optimizer = tf.keras.optimizers.Adam()
    start_epoch = load_checkpoint(self.tiers[tier_num], optimizer,path)
    train(self.tiers[tier_num], x, features,start_epoch, 5, batch_size,)


  def sample(self):
    pass
  
  def load_tiers(self, units, gmm, audio_length, sr , window): # Load tiers without weights
    self.cal_each_tier_dim(audio_length, sr ,window)
    self.tiers = []
    for i in range(len(self.time_splits)):
      if i == 0:
        self.tiers.append(Tier(units, self.tiers_f_dim[i], self.tiers_t_dim[i],gmm=gmm))
        self.tiers.append(Tier(units, self.tiers_f_dim[i + 1], self.tiers_t_dim[i], False, layers=5, gmm = gmm))

      elif (i+1) >= len(self.tiers_f_dim):
        self.tiers.append(Tier(units, self.tiers_f_dim[-1], self.tiers_t_dim[i], False, layers=5, gmm = gmm))
      else:
        self.tiers.append(Tier(units, self.tiers_f_dim[i], self.tiers_t_dim[i], False, layers=5, gmm = gmm))
        self.tiers.append(Tier(units, self.tiers_f_dim[i + 1], self.tiers_t_dim[i], False, layers=5, gmm =gmm))


  def process_data(self, dest_path = DESTINATION_PATH, data_dir = DATA_PATH, split_index=0,  batch_size = 8): #process data for just one split for training only to save storage space.
    # Calculate splits
    splits = get_splits(self.mel_splits, self.time_splits)

    # Make directories if not exists.
    if not os.path.exists(dest_path):
      os.mkdir(dest_path)
    
    for i in range((len(self.tiers) + 1),0,-1):
      tier_dir = os.path.join(dest_path, f'Tier {i}')
      sample_dir = os.path.join(dest_path, f'Tier {i}')
      if not os.path.exists(tier_dir):
        os.mkdir(tier_dir)
      if not os.path.exists(sample_dir):
        os.mkdir(sample_dir)

      if i != 1:
        feature_dir = os.path.join(dest_path, f'Tier {i}/features')
        if not os.path.exists(feature_dir):
          os.mkdir(feature_dir)
    
    file_list = os.listdir(data_dir)
    n_of_samples = len(file_list)
    batches = math.ceil(n_of_samples / batch_size) 
    start =  0
    stop = start+ batch_size

    # iterate to music samples in folder in batches and split at the particular split index to prepare data for training.
    for i in range(batches):
      data = np.zeros(shape=(batch_size, self.mel_dim, self.x_samples))
      filenames = file_list[start:stop]
      for j in file_names:
        # Load file
        with open(os.path.join(data_dir, j),'rb') as f:
          file_loaded = pickle.load(f)
        # Append to numpy array 'data'.
        data[j] = np.array(file_loaded)
      
      # Recursively split and save results in respective files.
      recursive_split_save(data,dest_path, splits[split_index:], filenames, self.tiers)
      start = stop
      stop = stop + batch_size

    


  

In [ ]:
model = MelNet()
for tiers in model.tiers:
  print(tiers.dims)

(32.0, 80.75)
(64.0, 80.75)
(64.0, 161.5)
(128.0, 161.5)
(128.0, 323.0)
(128.0, 646.0)
(128.0, 1292.0)


In [ ]:
# Review this code !!!!
from tqdm import tqdm

def interleave(base_tier, next_tier, axis=0):
  batch, dimf, dimt = base_tier.shape
  print(base_tier.shape)
  print(next_tier.shape)
  
  # If axis is 0, fuse along frequency axis
  if axis == 0:    
    fused_tier = np.zeros(shape=(batch, dimf*2, dimt))
    fused_tier[:,:dimf*2:2,:] = base_tier[:,:,:]
    fused_tier[:,1:dimf*2:2,:]  = next_tier[:,:,:]
  else:
    # fuse along time axis
    fused_tier = np.zeros(shape=(batch, dimf, dimt*2))
    fused_tier[:,:,:dimt*2:2] = base_tier[:,:,:]
    fused_tier[:,:,1:dimt*2:2] = next_tier[:,:,:]
  
  return fused_tier

def split(spectogram, axis=0):
  # Identify shape and extract length of both axis
  f , t = spectogram.shape[1:]

  # if split is along frequency axis
  if axis == 0:
    even = spectogram[:,1:f:2,:]
    odd = spectogram[:,:f:2,:]

  # else split along the time axis
  else:
    even = spectogram[:,:,1:t:2]
    odd = spectogram[:,:,:t:2]

  return even , odd



def load_dataset(filenames,feat_files=None, buffer_size=20, batch_size=16):
  if feat_files != None:
    dataset = tf.data.Dataset.from_tensor_slices((filenames, feat_files))
    has_feat = True
  else:
    dataset = tf.data.Dataset.from_tensor_slices((filenames))
    has_feat=False
    
  # Shuffle and batch
  dataset = dataset.shuffle(buffer_size).batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset, has_feat
  
def load_checkpoint(tier, optimizer, ckpt_path = "./checkpoints/train"):
  ckpt = tf.train.Checkpoint(tier,
                            optimizer = optimizer)
  ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=5)

  start_epoch = 0
  if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)
  return start_epoch

def train(tier, optimizer, file_train, feat_train = None, start_epoch=0, epochs=5, batch_size=16):
  dataset, has_feat = load_dataset(file_train, feat_train, batch_size)
  loss_plot = []
  num_steps = len(file_train) // batch_size
  for epoch in range(start_epoch, epochs):
      start = time.time()
      total_loss = 0

      if has_feat:
        for (batch, (spect, feat_spect)) in enumerate(dataset):
            batch_loss = train_step(tier, optimizer, spect, feat_spect)
            total_loss += batch_loss

            # if batch % 15 == 0:
            #     print ('Epoch {} Batch {} Loss {:.4f}'.format(
            #       epoch + 1, batch, batch_loss.numpy() / int(feat_spect.shape[1])))
                
      else:
        for (batch, spect) in tqdm(enumerate(dataset)):
            batch_loss = train_step(tier,optimizer, spect)
            total_loss += batch_loss

            if batch % 15 == 0:
                print ('Epoch {} Batch {} Loss {:.4f}'.format(
                  epoch + 1, batch, batch_loss.numpy() / int(feat_spect.shape[1])))

              
      # storing the epoch end loss value to plot later
      loss_plot.append(total_loss / num_steps)

      # if epoch % 5 == 0:
      #   ckpt_manager.save()

      print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                          total_loss/num_steps))
      print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

  return loss_plot


def train_step(tier,optimizer, x,features=None ):
  loss = 0
  with tf.GradientTape() as tape:
    if features == None:
      mu , std , alpha = tier(x)
    else:
      mu , std , alpha = tier(x,features)

    loss += gaussian_mixture_loss(x, mu, std, alpha)

    #total_loss = (loss / int(target.shape[1]))

  trainable_variables = tier.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))
  gc.collect()

  return loss #, total_loss






In [ ]:
import gc
dimf = 32
dimt= 32

batch = 4
#inputs = tf.random.normal((batch,dimf,dimt))
#tier = tf.random.normal((batch,dimf, dimt))
#print(inputs[0])
#input_t = tf.transpose(inputs,(0,2,1))
even , odd = split(tf.random.normal((300,dimf*2,dimt)), axis=0)
hidden_units = [64,128,256,512]
hidden_losses = []




#gc.collect()

In [ ]:
  hidden_losses

[]

In [ ]:
# for hidden in hidden_units:
#   lstm = Tier(hidden,dimf,dimt,layers=4,base_tier=False)
#   #lstm = TierAdd(512, dimf, dimt, layers = 3, base_tier=False)
#   #lstm_add = DelayedLSTMAdd(512,dimf,dimt)
#   #lstm(inputs,tier)
#   adam = tf.keras.optimizers.Adam()
#   loss = train(lstm, adam, even, odd,epochs=10, batch_size=16)
#   gc.collect()
#   hidden_losses.append(loss)

lstm2 = Tier(512,dimf,dimt,layers=4,base_tier=False)
adam = tf.keras.optimizers.Adam()
loss = train(lstm2, adam, even, odd,epochs=10, batch_size=16)
hidden_losses.append(loss)
